In [708]:
from getpass import getpass
import json
import requests
from datetime import datetime, timedelta
import time,os
import calendar
import os,glob
from os import path
from jira import JIRA
import csv
from os import path
import pandas as pd
from subprocess import Popen
from pick import pick


In [709]:
##Author(s): Yared Ayele, Maria Arbenina
##======================== MC API ==============================

#MC API URL 
API_URL = "https://mission-control-api.software.iceye.fi"
API_URL_EVENTS = "https://mission-control-api-events.software.iceye.fi"

now = datetime.now()
start_time = now.strftime("%Y-%m-%d") + "T00%3A00"

SO_USER = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
SO_PASSWORD = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

#CHANGE API_URL to API_URL_EVENTS if you need to run it for events (image activities before March 2022)
def get_token():
    # Login to Smooth Operator with legacy username and password
    username = SO_USER
    password = SO_PASSWORD
    r = requests.post(f'{API_URL}/api/token', data={
        "username": username,
        "password": password
    })
    if r.status_code != 200:
        print(f"{datetime.now()}: Login failed!")
        raise PermissionError()
    print(f"{datetime.now()}: Logged in!")
    return "Bearer " + r.json()['access']

## Login to Smooth Operator 
token = get_token()

2022-12-05 20:04:58.181483: Logged in!


In [710]:

#=================================== JIRA  ======================= 
user = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
apikey ="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
server = 'https://iceyedev.atlassian.net/'

options = {
 'server': server
}

jira = JIRA(options, basic_auth=(user,apikey) )

In [711]:
## API 
def lookAngleToIncidenceAngle(lookAngleDeg, sensorRadius, earthRadius):
    gamma = 180 - np.rad2deg(np.arcsin((sensorRadius) * np.sin(np.deg2rad(lookAngleDeg)) / earthRadius))
    theta_e = 180 - gamma - lookAngleDeg
    incidenceAngDeg = lookAngleDeg + theta_e
    return incidenceAngDeg


def reformat_time(time_string):
    return datetime.strptime(time_string, "%Y-%m-%dT%H:%M:%S%z").strftime("%Y-%m-%d %H:%M:%S")

def epoch_time(time_string):
    #format_time= reformat_time(time_string)
    pattern = "%Y-%m-%d %H:%M:%S%z"
    utc_epoch = calendar.timegm(time.strptime(time_string, pattern))
    utc_epoch = utc_epoch *1000
    return utc_epoch


## get a single Image/Activity 
def fetch_json(url, token):
    r = requests.get(url, headers={"Authorization": token})
    #print("." if r.status_code == 200 else f"error! {r.json()}\n", end="", flush=True)
    if r.status_code != 200:
        # Raise PermissionError()
        result = "error"
        print("fetch_json: PermissionError code " + str(r.status_code))
    else:
        result = r.json()
    return result

## get an Image Activity by 
def get_activity_byid(image_id,token):
    return fetch_json(
        f"{API_URL}/imaging_activities/{image_id}",token
    )

## get an Event by 
def get_event_byid(image_id,token):
    API_URL_EVENTS = "https://mission-control-api-events.software.iceye.fi"
    return fetch_json(
        f"{API_URL_EVENTS}/tasking/imageevents/{image_id}",token
    )

# COMMENT OUT THIS BLOCK TO UPDATE ONLY METADATA
## download images to a local folder
def downloadImages(imgID,imgType,dowloadloc): 
    os.chdir(dowloadloc)
    
    print("------------- Downloading QUICKLOOK: ic get "+str(imgID)+' quicklook --exclude "*kml"') 
    os.system('ic {} {} {} {}' .format(str('get'), str(imgID),str(imgType),str('--exclude "*kml"')))
    
    print("------------- Downloading metadata file for GRD: ic get "+str(imgID)+'grd --exclude "*tif"')
    os.system('ic {} {} {} {}' .format(str('get'), str(imgID),'grd',str('--exclude "*tif"')))
    
    print("------------- Downloading Auto QA results: ic get "+str(imgID)+' _ --include "ds*-qa-report.json"')
    os.system('ic {} {} {} {}' .format(str('get'), str(imgID),'_',str('--include "ds*-qa-report.json"')))
    
    print("--------------Download completed ---------------")



def update_imageMetadata():
    i=0
    # Set current Directory 
    currentDir = "C:\\Users\\marbenina\\ICEYEDOCS\\Python\\METADATA\\"
    
    # Fetch all fields
    allfields = jira.fields()
    # Make a map from field name -> field id
    nameMap = {field['name']:field['id'] for field in allfields}
    
    # Specify Jira filter
    queryFilter='REPORTED_Image_Activities'
    jql = jira.search_issues('filter='+queryFilter,startAt=0,maxResults=2000)
    
    for issue in jql:
        
        projectKey = issue.fields.project.key  
       
        i=i+1
        print("Reading a ticket searching for Image ID")
        fkey = issue.key 
        print("-------------"+str(fkey))
        task_id = getattr(issue.fields, nameMap['Task ID'])
        image_id = str(getattr(issue.fields, nameMap['Image ID'])) if str(projectKey) == "FOA" else str(getattr(issue.fields, nameMap['Return Image ID']))
        print("Image ID: " + str(image_id))
        # print(len(image_id))
        # if image_id is None or not image_id.isdigit():
            # image_id = 0
        
        #GET Custom Fields ID
        idtask_id = nameMap['Task ID']
        idimage_priority = nameMap['Image Priority']
        idlook_side = nameMap['Look side']
        idlook_angel = nameMap['Look Angle [deg]']
        idorbit_direction = nameMap['Orbit direction']
        idsensor_mode = nameMap['Sensor Mode']
        idacquisition_time= nameMap['Acquisition Time']
        idsatellite = nameMap['Satellite(s)']
        idtelemetry_link = nameMap['Telemetry link']
        idctivity_id= nameMap['Activity ID']
        idimage_id =  nameMap['Image ID']
        idmetadata_autoupdated = nameMap['Metadata auto-updated']
        idAOI_centerLatitude = nameMap['AOI Center Latitude']
        idAOI_centerLongitude = nameMap['AOI Center Longitude']
        look_side = getattr(issue.fields, nameMap['Look side'])
        idacq_status = nameMap['Acquisition status']

        
        # Get current value of Metadata auto-updated field 
        metadata_autoupdatedValue = getattr(issue.fields, nameMap['Metadata auto-updated'])
        print("Metadata auto-updated: " + str(metadata_autoupdatedValue))
               
        # Setting isActivity based on image ID
        if(str(metadata_autoupdatedValue) == 'No'):
            
            if(int(image_id) >= 300000):
                activity = get_activity_byid(image_id,token)
                isActivity = True
                
            else:
                event = get_event_byid(image_id,token)
                isActivity = False
                    
           
            #Create a JSON file from the Activity/Events outputs 
            #The Json file is image_id_metadata.json 
            with open(str(image_id)+'_metadata.json', 'w') as f:
                if(isActivity):
                    json.dump(activity, f, ensure_ascii=False)
                    print("Metadata.json for this activity is downloaded")
                else:
                    json.dump(event, f, ensure_ascii=False)
                    print("Metadata.json for event is downloaded")
            
            
            print("------------- Mapping metadata fields to Jira fields")
            
            error = activity if(isActivity) else event 
            #There is some sort of error when calling MC API 
            if(error == "error"):
                print("Some error when calling MC API")
                            
            else:
     
                try:
                    #status = activity['state'] if(isActivity) else event['status']['name']#Status
                    task_id = activity['task_id'] if(isActivity) else event['task_id']#Task ID
                    activity_id = activity['id'] if(isActivity) else event['id']#Activity ID / Image ID
                    satellite = activity['satellite'] if(isActivity) else event['satellite']['name'] #Satellite name
                    image_priority = activity['task_priority'] if(isActivity) else event['priority'] #Image Priority 
                    sensor_mode = activity['mode'] if(isActivity) else event['sensor'] #Sensor Mode
                    imaging_start =activity['imaging_start'] if(isActivity) else event['start'] #Imaging Start
                    duration =activity['duration'] if(isActivity) else event['duration'] #Duration
                    
                    look_angel = activity['look_angle'] if(isActivity) else event['look_angle'] #Sensor Mode
                    center_lat = activity['center_lat'] if(isActivity) else event['center_lat'] #enter_lat
                    center_lon = activity['center_lon'] if(isActivity) else event['center_lon'] #center_lon
                    acq_status = activity['state'] if(isActivity) else event['status']['name'] #acquisition status
                    
                    if(isActivity):
                        look_side = activity['input_file']['metadata_json']['look_side'] #Look Side
                        orbit_direction = activity['input_file']['metadata_json']['orbit_direction'] #Look Side
                    
                    else:
                        # metadata_json = event['input_file']['metadata_json'] #Look Side
                        # metadata_json = json.loads(metadata_json)
                        # orbit_direction = metadata_json['orbit_direction']
                        
                        #UNCOMMENT FOR ACTIVITIES
                        orbit_direction = event['pass_type']
                        look_side = event['input_file']['metadata_json']['look_side']
                
                
                    #Look Side and orbit direction format
                    #UNCOMMENT FOR ACTIVITIES
                    
                    look_side = look_side.replace('left','Left')
                    look_side = look_side.replace('right','Right')
                    orbit_direction = orbit_direction.replace('ASCENDING','Ascending')
                    orbit_direction = orbit_direction.replace('DESCENDING','Descending')
                    
                    #Sensor Modes (MC has an underscore)
                    
                    sensor_mode = sensor_mode.replace('_',' ')
                    sensor_mode = sensor_mode.replace('extended','Extended')
                    sensor_mode = sensor_mode.replace('range','Range')
                    sensor_mode = sensor_mode.replace('dwel','Dwel')
                    sensor_mode = sensor_mode.replace('area','Area')
                    sensor_mode = sensor_mode.replace('high','HIGH')
                    sensor_mode = sensor_mode.replace('multi','Multi')
                    #mode names
                    sensor_mode = sensor_mode.replace('spotlight','Spotlight')
                    sensor_mode = sensor_mode.replace('stripmap','Stripmap')
                    sensor_mode = sensor_mode.replace('spot','Spot')
                    sensor_mode = sensor_mode.replace('topSar','TopSAR')
                    sensor_mode = sensor_mode.replace('topsar','TopSAR')
                    sensor_mode = sensor_mode.replace('Topsar','TopSAR')
                    sensor_mode = sensor_mode.replace('Stripmap Extended','Stripmap Extended Range')
                    
                    
                    sensor_mode = sensor_mode.replace("(left)","")
                    sensor_mode = sensor_mode.replace("(right)","")
                    sensor_mode = sensor_mode.replace("(LEFT)","")
                    sensor_mode = sensor_mode.replace("(RIGHT)","")
                    sensor_mode = sensor_mode.strip()
                    
                
                    #Format Start and End Dates 
                    telemetry_start =  datetime.strptime(imaging_start, "%Y-%m-%dT%H:%M:%S%z")
                    telemetry_start = telemetry_start - timedelta(minutes = 10)
                    telemetry_end = telemetry_start + timedelta(minutes = int(duration) +10) 
                    telemetry_link =  f"https://telemetry.product.iceye.fi/d/iceyedsds/key-parameters?orgId=1&from={epoch_time(str(telemetry_start))}&to={epoch_time(str(telemetry_end))}&var-satellite={satellite.upper()}&var-Resolution=10s" 
                
                    print("------------- Updating Jira Fields with image meta data")
                    issue.update(fields={
                                     idctivity_id: str(activity_id), 
                                     idtask_id: str(task_id), 
                                     idsatellite: [{'value':satellite.upper()}], 
                                     idimage_priority: int(image_priority),
                                     idsensor_mode: {'value':str(sensor_mode)},
                                     idacquisition_time: imaging_start, ##This is being convereted  by Jira to Finnish Time
                                     idlook_angel: float(look_angel),
                                     idlook_side: {'value':str(look_side)},
                                     idorbit_direction: {'value':str(orbit_direction)},
                                     idtelemetry_link: str(telemetry_link),
                                     idmetadata_autoupdated: {'value':'Yes'},
                                     idAOI_centerLatitude: float(center_lat),
                                     idAOI_centerLongitude: float(center_lon),
                                     idacq_status: str(acq_status)
                                    })
                    print("------------- Metadata fields updated")
                    
                #Download images, metadata, and auto QA results 
                    #if 'dowloadloc' in locals():
                        #os.remove(dowloadloc)  
                    dowloadloc = currentDir + "downloads\img_" +str(image_id)
                    os.makedirs(dowloadloc)
                    downloadImages(image_id,"QUICKLOOK",dowloadloc)
                
                #Attach files
                    print("------------- Attaching QUICKLOOK ")
                    jpgFilenamesList = glob.glob('*'+str(image_id)+'*''.PNG')
                    for i in range(len(jpgFilenamesList)):
                    #print(jpgFilenamesList[i])
                        imgFile = jpgFilenamesList[i]
                        file_obj = open(imgFile,'rb')
                    #print(file_obj)
                        jira.add_attachment(issue=issue.key, attachment=file_obj)
                
                    print("------------- Attaching XML file")
                    xmlFilenamesList = glob.glob('*'+str(image_id)+'*''.XML')
                    for i in range(len(xmlFilenamesList)):
                    #print(xmlFilenamesList[i])
                        xmlFile =xmlFilenamesList[i]
                        xmlfile_obj = open(xmlFile,'rb')
                    #print(xmlfile_obj)
                        jira.add_attachment(issue=issue.key, attachment=xmlfile_obj)
                        
                    print("------------- Attaching Auto QA .json")
                    jsonFilenamesList = glob.glob('*'+str(image_id)+'*''.JSON')
                    for i in range(len(jsonFilenamesList)):
                    #print(jsonFilenamesList[i])
                        jsonFile =jsonFilenamesList[i]
                        jsonfile_obj = open(jsonFile,'rb')
                    #print(jsonfile_obj)
                        jira.add_attachment(issue=issue.key, attachment=jsonfile_obj)   
                
                    
                           
                       
                # Add comment to FOA ticket
                    jira.add_comment(fkey,'Script Automation: image metadata fields updated, quicklook and auto QA results attached (if exist in AWS).')
                
                except Exception as e:
                    print("==========Exception happened: skipping "+str(fkey)+ "==========" + str(e))
                    pass
                
                



In [712]:
update_imageMetadata()